In [32]:
from pysat import solvers
from itertools import combinations
from itertools import product

Biến logic cho ô [x, y] của matrix(m, n) có giá trị tuyệt đối là x * n + y + 1. Nếu dương là Trap, ngược là lại Gem

Dễ dàng chứng minh được rằng: <br >
Ít nhất m literals trong n literals là đúng khi và chỉ khi với mỗi bộ hoán vị có m - n + 1 literals, luôn có ít nhất 1 literal đúng

In [33]:
def read_matrix(file_name):
    with open(file_name) as f:
        matrix = []
        for line in f:
            matrix.append([int(x) if x != '_' else x for x in line.strip().split()])
    return matrix

# Generate a clause that is true if and only if exactly n literals of the literals are true
def generate_cnf_exactly_n(m, literals):
    cnf = []
    n = len(literals)

    #At least n of the literals are true
    for comb in combinations(literals, n - m + 1):
        cnf.append(list(comb))

    #At most n of the literals are true 
    #Equivalent to at least n-m+1 of the literals are false
    for comb in combinations(literals, m+1):
        cnf.append([-lit for lit in comb])

    return cnf

generate_cnf_exactly_n(2, [1, 2, 3, 4, 5])
# Get no. cell of Trap and Gem around a cell
def get_around(matrix, i, j):
    m = len(matrix)
    n = len(matrix[0])
    move = [[-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 1], [1, -1], [1, 0], [1, 1]]
    around = []
    for [x, y] in move:
        if 0 <= i + x < m and 0 <= j + y < n and matrix[i + x][j + y] == '_':
            around.append((i + x) * n + (j + y) + 1)

    return around


In [34]:
def generate_cnf(matrix):
    m = len(matrix)
    n = len(matrix[0])
    cnf = []

    for i in range(m):
        for j in range(n):
            if matrix[i][j] != '_':
                around = get_around(matrix, i, j)
                cnf_cell = generate_cnf_exactly_n(matrix[i][j], around)
                
                for clause in cnf_cell:
                    if clause not in cnf:
                        cnf.append(clause)

    return cnf

In [35]:
def solve_cnf_pysat(cnf):
    solver = solvers.Glucose3()
    for clause in cnf:
        solver.add_clause(clause)

    if solver.solve():
        model = solver.get_model()
        return model
    
    return None

In [36]:
# sat-solver dpll algorithm
def solve_cnf_my_solution(cnf):
    def convert_to_literal(cnf):
        literal_cnf = []
        for clause in cnf:
            literal_clause = set()
            for var in clause:
                literal_clause.add((str(abs(var)), var > 0))
            literal_cnf.append(literal_clause)
        return literal_cnf
    def __select_literal(cnf):
        for c in cnf:
            for literal in c:
                return literal[0]
 
    def dpll(cnf, assignments={}): 
        if len(cnf) == 0:
            return True, assignments
    
        if any([len(c)==0 for c in cnf]):
            return False, None
    
        l = __select_literal(cnf)
    
        new_cnf = [c for c in cnf if (l, True) not in c]
        new_cnf = [c.difference({(l, False)}) for c in new_cnf]
        sat, vals = dpll(new_cnf, {**assignments, **{l: True}})
        if sat:
            return sat, vals
    
        new_cnf = [c for c in cnf if (l, False) not in c]
        new_cnf = [c.difference({(l, True)}) for c in new_cnf]
        sat, vals = dpll(new_cnf, {**assignments, **{l: False}})
        if sat:
            return sat, vals
        return False, None
    
    cnf = convert_to_literal(cnf)
    result, assignments = dpll(cnf)
    if result:
       return [int(var) if assignments[var] else -int(var) for var in assignments]
    return None


In [37]:
def solve_cnf_brute_force(cnf):
    variables = set(abs(var) for clause in cnf for var in clause)
    n = len(variables)
    all_possible_assignments = product([True, False], repeat=n)

    for assignment in all_possible_assignments:
        satisfies_all_clauses = True
        # Tạo một từ điển ánh xạ biến đến giá trị Boolean của chúng trong phân bổ hiện tại
        assignment_dict = {var: val for var, val in zip(sorted(variables), assignment)}
        for clause in cnf:
            satisfies_clause = False
            for literal in clause:
                # Kiểm tra nếu biến hoặc phủ định của biến đúng với phân bổ hiện tại
                if literal > 0:
                    if assignment_dict[abs(literal)]:
                        satisfies_clause = True
                        break
                else:
                    if not assignment_dict[abs(literal)]:
                        satisfies_clause = True
                        break
            
            # Nếu bất kỳ clause nào không được thỏa mãn, ngay lập tức hủy bỏ phân bổ này
            if not satisfies_clause:
                satisfies_all_clauses = False
                break

        # Nếu phân bổ này thỏa mãn tất cả các clause, trả về nó
        if satisfies_all_clauses:
            # Chuyển đổi phân bổ sang định dạng đầu ra thích hợp
            return[var if val else -var for var, val in assignment_dict.items()]
            

    return None


In [38]:
def solve_cnf_back_tracking(cnf):
    variables = list(set(abs(var) for clause in cnf for var in clause))
    variables.sort()

    def is_satisfied(clause, assignment):
    # Kiểm tra xem clause có thỏa mãn với assignment hiện tại không
        for var in clause:
            if (var > 0 and assignment.get(var, False)) or (var < 0 and not assignment.get(-var, True)):
                return True
        return False

    def backtrack(assignment):
        # Kiểm tra nếu tất cả các biến đều đã được gán
        if len(assignment) == len(variables):
            # Nếu tất cả clauses thỏa mãn, trả về assignment
            if all(is_satisfied(clause, assignment) for clause in cnf):
                return assignment
            else:
                return None

        # Lấy biến tiếp theo chưa được gán
        for var in variables:
            if var not in assignment and -var not in assignment:
                # Thử gán True, sau đó False cho biến này
                assignment[var] = True
                if backtrack(assignment) is not None:
                    return assignment
                assignment[var] = False
                if backtrack(assignment) is not None:
                    return assignment
                del assignment[var]
                break
        return None
    # Khởi tạo quá trình backtrack với assignment rỗng
    return [var if val else -var for var, val in backtrack({}).items()]

In [39]:
import copy
def write_solution(model, matrix, output_file):
    matrix_copy = []
    
    if model is not None:
        matrix_copy = copy.deepcopy(matrix)
        m = len(matrix_copy[0])
        for x in model:
            y = x if x > 0 else -x
            y -= 1
            if matrix_copy[y // m][y % m] == '_':
                matrix_copy[y // m][y % m] = 'T' if x > 0 else 'G'
    
    with open(output_file, 'w') as f:
        for row in matrix_copy:
            f.write(' '.join(str(x) for x in row) + '\n')

In [40]:
import signal

class TimeoutError(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutError("Timed out")

def run_with_timeout(func, args=(), kwargs={}, timeout_duration=5):
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout_duration)
    try:
        result = func(*args, **kwargs)
    finally:
        signal.alarm(0) # Cancel the alarm
    return result

In [45]:
import os
import time

folder_path_input = 'testcases/input'

folder_path_output = ['output-pysat', 'output-my-solution', 'output-brute-force', 'output-back-tracking']
functions_solve_cnf = [solve_cnf_pysat, solve_cnf_my_solution, solve_cnf_brute_force, solve_cnf_back_tracking]

num_functions = len(functions_solve_cnf)
time_out = 180

for index, file_name in enumerate(os.listdir(folder_path_input)):
    print(f"Test {index}: {file_name}")
    matrix = read_matrix(os.path.join(folder_path_input, file_name))
    cnf = generate_cnf(matrix)

    for i in range(num_functions):
        print(f"{functions_solve_cnf[i].__name__}: ", end="")
        
        try:
            start_time = time.time()    
            model = run_with_timeout(functions_solve_cnf[i], args=(cnf,), timeout_duration=time_out)
            end_time = time.time()
            print(f"{((end_time - start_time) * 1000):.8f}ms")
        except TimeoutError:
            print(f"Timed out ({time_out}s)")

        write_solution(model, matrix,
                        os.path.join("testcases", folder_path_output[i], f"output{file_name[5:-4]}.txt"))
    print()

Test 0: input0.txt
solve_cnf_pysat: 0.20146370ms
solve_cnf_my_solution: 0.30946732ms
solve_cnf_brute_force: 0.25582314ms
solve_cnf_back_tracking: 0.31971931ms

Test 1: input5.txt
solve_cnf_pysat: 0.96130371ms
solve_cnf_my_solution: 112.07985878ms
solve_cnf_brute_force: Timed out (60s)
solve_cnf_back_tracking: Timed out (60s)

Test 2: input4.txt
solve_cnf_pysat: 0.34236908ms
solve_cnf_my_solution: 58.22467804ms
solve_cnf_brute_force: Timed out (60s)
solve_cnf_back_tracking: Timed out (60s)

Test 3: input1.txt
solve_cnf_pysat: 0.11038780ms
solve_cnf_my_solution: 0.36811829ms
solve_cnf_brute_force: 0.89168549ms
solve_cnf_back_tracking: 0.80060959ms

Test 4: input3.txt
solve_cnf_pysat: 0.19168854ms
solve_cnf_my_solution: 2.76231766ms
solve_cnf_brute_force: Timed out (60s)
solve_cnf_back_tracking: Timed out (60s)

Test 5: input2.txt
solve_cnf_pysat: 0.27656555ms
solve_cnf_my_solution: 2.34460831ms
solve_cnf_brute_force: 18182.41429329ms
solve_cnf_back_tracking: 12701.00188255ms

